<h1><center><font size = 10> Assignment 2 </font></center></h1>
<h1><center><font size = 5> Author: Zach Chase </font></center></h1>

In [ ]:
from sklearn.datasets import load_diabetes
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Part 1
Data Exploration. Load the scikit-learn diabetes bunchobject into a variable. Print out the description of the dataset. Load the diabetesfeatures into a pandas dataframe with the proper column names. Add thetarget
variable to this same dataframe. Run a command to look at the data typesof
your dataframe to see if there is any missing data. Perform descriptive statisticson the numeric columns of your dataframe. Plot histograms of your datatoget afeel for each column’s distribution. Split your dataframe into a training andtest
set with 20% of your data being in the test set. Define a correlation matrix. Lookat values highly correlated with the target. Plot the correlation matrix withaSeaborn heatmap. Use a Seaborn pairplot to look at the scatter plots of thethreevalues with the highest target correlation. Prepare a feature set by droppingthetarget from your training dataframe. Copy your training target into a newdataframe.

In [ ]:
# Load teh data
diabetes = load_diabetes()

# Print out the description
diabetes.DESCR

In [ ]:
# Load the data into a Pandas DataFrame
df = pd.DataFrame(diabetes.data, columns = diabetes.feature_names)
df['progression'] = diabetes.target # Add target variable
df

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
fig = plt.figure(figsize = (15,20))
ax = fig.gca()
df.hist(ax = ax)
plt.show()

In [ ]:
X = df.drop(['progression'], axis = 1).to_numpy()
y = df['progression'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 8)

In [ ]:
df.corr()

In [ ]:
sns.heatmap(df.corr())
plt.show()

The values with large correlations with the target variable are bmi, s5 and bp.

In [ ]:
sns.pairplot(df[['bmi', 's5', 'bp', 'progression']])
plt.show()

In [ ]:
df_data = df.drop(["progression"], axis = 1)

# Part 2
Model Training. Train a linear regression model usingyour training set. Print the RMSE of your regression model on your trainingset.
Implement a cross_val_score on a decision tree regressor on your trainingset. Print out root mean and standard deviation of the cross-validation scores. Dothesame for a RandomForestRegressor. Record which model performs better

In [ ]:
lr = LinearRegression().fit(X_train, y_train)
mean_squared_error(lr.predict(X_train), y_train, squared=False)

In [ ]:
dt = DecisionTreeRegressor(random_state = 0)
scores = cross_val_score(dt, X_train, y_train, scoring='neg_mean_squared_error')
tree_rmse_scores = np.sqrt(-scores)
print('Scores:', tree_rmse_scores)
print('Mean:', tree_rmse_scores.mean())
print('Standard Deviation:', tree_rmse_scores.std())

In [ ]:
rf = RandomForestRegressor(random_state = 0)
scores = cross_val_score(rf, X_train, y_train, scoring='neg_mean_squared_error')
forest_rmse_scores = np.sqrt(-scores)
print('Scores:', forest_rmse_scores)
print('Mean:', forest_rmse_scores.mean())
print('Standard Deviation:', forest_rmse_scores.std())

Note that the linear regression model had a rmse of 53.5. Of the two other methods, the random forest regressor performed better with an rmse of 57.2, compared to the decision tree regressor which has an rmse of 75.6.

# Part 3
Model Tuning. Print out the parameters of your randomforest model. Do a grid search cross-validation with the following values:
n_estimators: 3,10,30 and max_features: 2,4,6,8, as well as the followingexperiment: bootstrap: False, n_estimators: 3,10 and max_features: 2,3,4. Print
out the best parameters and the best performing model based on this gridsearch. Using the cv_results dictionary, print out the rmse of each feature combinationforcomparison. Also print out the feature importances of the best performinggridsearch model. Describe how it compares with the correlation matrix we
implemented earlier.

In [ ]:
rf.get_params()

In [ ]:
param_grid = [{'n_estimators': [3,10,30], 'max_features':[2,4,6,8]}, 
              {'bootstrap':[False], 'n_estimators':[3,10], 'max_features':[2,3,4]}]

gridsearch = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
gridsearch.fit(X_train, y_train)

In [ ]:
gridsearch.best_params_

In [ ]:
gridsearch.best_estimator_

In [ ]:
for mean_score, params in zip(gridsearch.cv_results_['mean_test_score'], gridsearch.cv_results_['params']):
    print(np.sqrt(-mean_score), params)

In [ ]:
feature_importance = gridsearch.best_estimator_.feature_importances_
sorted(zip(feature_importance, df.drop(['progression'], axis = 1)), reverse = True)

Note that the values we looked at earlier with the largest correlation values with our target variable were bmi, s5, and bp. These variables are 3 of the 4 most important according to our random forest model.

# Part 4
Model Evaluation. Document the best-performingmodel
between the single feature model you trained in Assignment 1, and the modelsyou trained in part 2 and 3 of this assignment. Evaluate the best performingmodel against your test set. Save your model for future use.

In [ ]:
# Load Data
diabetes_data = load_diabetes()
X_bmi = diabetes_data.data
y_bmi = diabetes_data.target

X_train_bmi = X_bmi[:-20,2].reshape(-1,1)
X_test_bmi = X_bmi[:20,2].reshape(-1,1)
y_train_bmi = y_bmi[:-20].reshape(-1,1)
y_test_bmi = y_bmi[:20].reshape(-1,1)

diabetes_lm_bmi = LinearRegression().fit(X_train_bmi, y_train_bmi)

value = mean_squared_error(y_train_bmi, diabetes_lm_bmi.predict(X_train_bmi), squared = False)
print("Assignment 1 RMSE with train data using BMI only: ", value)

In [ ]:
value = min(np.sqrt(-gridsearch.cv_results_['mean_test_score']))
print("Random Forest RMSE with train data: ", value)

In [ ]:
value = mean_squared_error(lr.predict(X_train), y_train, squared=False)
print("Linear Regression RMSE with train data: ", value)

In [ ]:
value_bmi = mean_squared_error(diabetes_lm_bmi.predict(X_test[:,2].reshape(-1,1)), y_test, squared = False)
value_lr = mean_squared_error(lr.predict(X_test), y_test, squared=False)
value_rf = mean_squared_error(gridsearch.best_estimator_.predict(X_test), y_test, squared = False)

print("Linear Regression with BMI on test data RMSE: ", value_bmi)
print("Linear Regression with all on test data RMSE: ", value_lr)
print("Random Forest on test data RMSE: ", value_rf)

From these results it appears the model that performs best at minimizing RMSE is the Linear Regression model using all of the features. On the training data it has a RMSE of 53.52, and on the test data it has a RMSE of 54.20.

In [ ]:
import joblib
joblib.dump(lr, "my_lr_model.pkl")

In [ ]:
!ls my_lr_model.*